# Grid simulator for Simulation Fitting Algorithms

In [1]:
import os
os.chdir(f"../../..")
from SOURCE.CLASS_CODE_GPU_Classes import *
from SOURCE.CLASS_CODE_Image_Manager import *
from SOURCE.CLASS_CODE_Polarization_Obtention_Algorithms import Gradient_Algorithm
import numpy as n/home/melanie/Desktop/Conical_Refraction_Polarimeter/OUTPUTp
import json
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from IPython import display
from time import time

In [2]:
# Set the PARAMETERS ############################################
##################################################################
experiment_name="Basler_like_R0_158_w0_20_3th_decimal"
output_directory=f"./OUTPUT/LIBRARIES_OF_THEORETICAL_CR/{experiment_name}/"
#os.chdir(f"/home/melanie/Desktop/Conical_Refraction_Polarimeter/")

randomization_seed=666
image_depth=8 # or 16 bit per pixel
image_shortest_side=int(540/3)
saturation=1
significant_decimal=3

# 1. SIMULATION ####################################################
# Ring parameters to test (each will be a different simulation)
phiCR_s=np.linspace(-180,180,360*10**significant_decimal+1)*np.pi/180
R0_s=np.array([158])/3 #np.linspace(70,180,40) # in pxels 153
w0_s=np.array([20])/3 #np.linspace(8,50,40) 11
rho_0s=R0_s/w0_s


resolution_side_nx=image_shortest_side # generated images will be resolution_side x resolution_side
# Other parameters
max_k=50
num_k=1200
sim_chunk_ax=image_shortest_side


# 4. GRAVICENTER iX and PROFILES ######################################
X=int(resolution_side_nx*1.4/2)


##################################################
##################################################

# General considerations
image_directory=f"{output_directory}/SIMULATIONS/ddeg_{(1/10**significant_decimal):.{significant_decimal}}_nx_{image_shortest_side}_iX_{X}_depth_{image_depth}_sat_{saturation}/"
os.makedirs(image_directory, exist_ok=True)
im_type=np.uint16 if image_depth==16 else np.uint8
max_intensity=65535 if image_depth==16 else 255
np.random.seed(randomization_seed)

In [3]:
# GENERAL ROUTINES #################################
def compute_intensity_gravity_center(image):
    """
        Expects input image to be an array of dimensions [h, w].
        It will return an array of gravity centers [2(h,w)] in pixel coordinates
        Remember that pixel coordinates are set equal to numpy indices

    """
    # image wise total intensity and marginalized inensities for weighted sum
    intensity_in_w = np.sum(image, axis=0) # weights for x [raw_width]
    intensity_in_h = np.sum(image, axis=1) # weights for y [raw_height]
    total_intensity = intensity_in_h.sum()

    # Compute mass center for intensity
    # [2] (h_center,w_center)
    return np.nan_to_num( np.stack(
        (np.dot(intensity_in_h, np.arange(image.shape[0]))/total_intensity,
         np.dot(intensity_in_w, np.arange(image.shape[1]))/total_intensity)
        ) )

def compute_raw_to_centered_iX(image, X):

    g_raw = compute_intensity_gravity_center(image)
    # crop the iamges with size (X+1+X)^2 leaving the gravity center in
    # the central pixel of the image. In case the image is not big enough for the cropping,
    # a 0 padding will be made.
    centered_image = np.zeros( (2*X+1, 2*X+1),  dtype = image.dtype )

    # we round the gravity centers to the nearest pixel indices
    g_index_raw = np.rint(g_raw).astype(int) #[N_images, 2]

    # obtain the slicing indices around the center of gravity
    # TODO -> make all this with a single array operation by stacking the lower and upper in
    # a new axis!!
    # [ 2 (h,w)]
    unclipped_lower = g_index_raw[:]-X
    unclipped_upper = g_index_raw[:]+X+1
    # unclippde could get out of bounds for the indices, so we clip them
    lower_bound = np.clip( unclipped_lower, a_min=0, a_max=image.shape)
    upper_bound = np.clip( unclipped_upper, a_min=0, a_max=image.shape)
    # we use the difference between the clipped and unclipped to get the necessary padding
    # such that the center of gravity is left still in the center of the image
    padding_lower = lower_bound-unclipped_lower
    padding_upper = upper_bound-unclipped_upper

    # crop the image
    centered_image[padding_lower[0]:padding_upper[0] or None,
                                    padding_lower[1]:padding_upper[1] or None ] = \
                  image[lower_bound[0]:upper_bound[0],
                                      lower_bound[1]:upper_bound[1]]
    return centered_image
    '''
    else:
        # We compute the center of gravity of the cropped images, if everything was made allright
        # they should get just centered in the central pixels number X+1 (index X)
        g_centered = compute_intensity_gravity_center(centered_image)

        # We now compute a floating translation of the image so that the gravicenter is exactly
        # centered at pixel (607.5, 607.5) (exact center of the image in pixel coordinates staring
        # form (0,0) and having size (607*2+1)x2), instead of being centered at the beginning of
        # around pixel (607,607) as is now
        translate_vectors = X+0.5-g_centered #[ 2(h,w)]
        T = np.float64([[1,0, translate_vectors[1]], [0,1, translate_vectors[0]]])
        return cv2.warpAffine( centered_image, T, (X*2+1, X*2+1),
                    flags=interpolation_flag) # interpolation method
    '''

In [4]:
# Initialize the vigilant
try:
    phase_vigilant = json.load(open(f"{output_directory}/STRUCTURE_Grid.json"))
except:
    phase_vigilant = {'R0s':[], 'w0s':[], 'phiCRs':[], 'IDs':[], 'rel_path':[]}

# Set the objects ready ##################
# The simulator object
simulator=RingSimulator_Optimizer_GPU( n=1.5, a0=1.0, max_k=max_k, num_k=num_k, nx=resolution_side_nx, sim_chunk_x=sim_chunk_ax, sim_chunk_y=sim_chunk_ax)

# Execute the stuff #####################
i=1
total=phiCR_s.shape[0]*R0_s.shape[0]*w0_s.shape[0]
elapsed=0
beg=time()
output_every=100000

for phi_CR in phiCR_s:
    for R0 in R0_s:
        for w0 in w0_s:
            ID=f"phiCR_{phi_CR}_R0_{R0}_w0_{w0}"
            if ID not in phase_vigilant['IDs']:
                # simulate image
                image=simulator.compute_CR_ring( CR_ring_angle=phi_CR, R0_pixels=R0, Z=0, w0_pixels=w0)
                
                # convert to selected uint format
                image = (max_intensity*(image/image.max())).astype(im_type)
                
                # get iX (saturated) image
                image = np.where( image<=(max_intensity*saturation), image, max_intensity*saturation) # saturation application if necessary
                image = compute_raw_to_centered_iX(image, X)
                
                # save the image
                rel_path=f"{image_directory}/{ID}.png"
                #t=time()
                cv2.imwrite( rel_path, image)
                #print(f"Saved {i} in {time()-t} s size {os.path.getsize(rel_path)}")
                #t=time()
                #im=cv2.imread(rel_path, cv2.IMREAD_ANYDEPTH)
                #print(f"Loaded {i} in {time()-t} s")

                if image is None:
                    raise ValueError

                
                #append the data
                phase_vigilant['IDs'].append(ID)
                phase_vigilant['R0s'].append(float(R0))
                phase_vigilant['phiCRs'].append(float(phi_CR))
                phase_vigilant['w0s'].append(float(w0))
                phase_vigilant['rel_path'].append(rel_path)
                
                if i%output_every==0:
                    # we save the progess (in order to be able to quit and resume)
                    json.dump(phase_vigilant, open( f"{output_directory}/STRUCTURE_Grid.json", "w"))
                    display.clear_output(wait=True)
                    elapsed=time()-beg
                    print(f"["+'#'*(int(100*i/total))+' '*(100-int(100*i/total))+f"] {100*i/total:3.4}% \n\nSimulated: {i}/{total}\nElapsed time: {elapsed//3600} h {elapsed//60-(elapsed//3600)*60} min {elapsed-(elapsed//60)*60-(elapsed//3600)*60:2.4} s")
            i+=1

2021-12-09 17:43:48.285206: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:113] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You do not need to update to CUDA 9.2.88; cherry-picking the ptxas binary is sufficient.
2021-12-09 17:43:48.309393: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:113] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You do not need to update to CUDA 9.2.88; cherry-picking the ptxas binary is sufficient.
2021-12-09 17:43:48.331978: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:113] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, 

2021-12-09 17:43:50.942382: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:113] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You do not need to update to CUDA 9.2.88; cherry-picking the ptxas binary is sufficient.
2021-12-09 17:43:51.028952: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:113] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You do not need to update to CUDA 9.2.88; cherry-picking the ptxas binary is sufficient.
2021-12-09 17:43:51.039830: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:113] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, 

Saved 1 in 0.0004050731658935547 s size 6778
Loaded 1 in 0.0022945404052734375 s
Saved 2 in 0.00030517578125 s size 6778
Loaded 2 in 0.00030517578125 s
Saved 3 in 0.00024366378784179688 s size 6777
Loaded 3 in 0.00032830238342285156 s
Saved 4 in 0.0002415180206298828 s size 6783
Loaded 4 in 0.0002970695495605469 s
Saved 5 in 0.0002872943878173828 s size 6784
Loaded 5 in 0.000301361083984375 s
Saved 6 in 0.00037407875061035156 s size 6784
Loaded 6 in 0.0003752708435058594 s
Saved 7 in 0.0003554821014404297 s size 6781
Loaded 7 in 0.0003724098205566406 s
Saved 8 in 0.0006139278411865234 s size 6780
Loaded 8 in 0.0004642009735107422 s
Saved 9 in 0.00042629241943359375 s size 6782
Loaded 9 in 0.00047087669372558594 s
Saved 10 in 0.0003876686096191406 s size 6782
Loaded 10 in 0.00044035911560058594 s
Saved 11 in 0.0004127025604248047 s size 6782
Loaded 11 in 0.00045871734619140625 s
Saved 12 in 0.00035762786865234375 s size 6782
Loaded 12 in 0.0004875659942626953 s
Saved 13 in 0.00042796134

Loaded 134 in 0.000347137451171875 s
Saved 135 in 0.0003418922424316406 s size 6778
Loaded 135 in 0.00037598609924316406 s
Saved 136 in 0.00032901763916015625 s size 6778
Loaded 136 in 0.0003695487976074219 s
Saved 137 in 0.0003726482391357422 s size 6779
Loaded 137 in 0.0003542900085449219 s
Saved 138 in 0.00028061866760253906 s size 6778
Loaded 138 in 0.00033974647521972656 s
Saved 139 in 0.000274658203125 s size 6778
Loaded 139 in 0.00032901763916015625 s
Saved 140 in 0.00028443336486816406 s size 6777
Loaded 140 in 0.00033020973205566406 s
Saved 141 in 0.00027108192443847656 s size 6777
Loaded 141 in 0.00032901763916015625 s
Saved 142 in 0.0002770423889160156 s size 6782
Loaded 142 in 0.0003273487091064453 s
Saved 143 in 0.00028896331787109375 s size 6781
Loaded 143 in 0.0003535747528076172 s
Saved 144 in 0.00028443336486816406 s size 6781
Loaded 144 in 0.00033164024353027344 s
Saved 145 in 0.00030303001403808594 s size 6781
Loaded 145 in 0.0003342628479003906 s
Saved 146 in 0.0002

Saved 277 in 0.00030517578125 s size 6786
Loaded 277 in 0.00036215782165527344 s
Saved 278 in 0.00039458274841308594 s size 6788
Loaded 278 in 0.0003757476806640625 s
Saved 279 in 0.00032973289489746094 s size 6788
Loaded 279 in 0.00038552284240722656 s
Saved 280 in 0.0005428791046142578 s size 6789
Loaded 280 in 0.00043773651123046875 s
Saved 281 in 0.00048065185546875 s size 6789
Loaded 281 in 0.0003676414489746094 s
Saved 282 in 0.00031065940856933594 s size 6789
Loaded 282 in 0.00035881996154785156 s
Saved 283 in 0.00029969215393066406 s size 6788
Loaded 283 in 0.0003688335418701172 s
Saved 284 in 0.000396728515625 s size 6790
Loaded 284 in 0.0004360675811767578 s
Saved 285 in 0.0003597736358642578 s size 6791
Loaded 285 in 0.0003695487976074219 s
Saved 286 in 0.0003733634948730469 s size 6791
Loaded 286 in 0.00037407875061035156 s
Saved 287 in 0.0003216266632080078 s size 6789
Loaded 287 in 0.0004012584686279297 s
Saved 288 in 0.0003192424774169922 s size 6789
Loaded 288 in 0.0003

...

Saved 855 in 0.00026917457580566406 s size 6794
Loaded 855 in 0.00031185150146484375 s
Saved 856 in 0.00038504600524902344 s size 6794
Loaded 856 in 0.00035262107849121094 s
Saved 857 in 0.00033020973205566406 s size 6796
Loaded 857 in 0.00037097930908203125 s
Saved 858 in 0.00034356117248535156 s size 6798
Loaded 858 in 0.0003440380096435547 s
Saved 859 in 0.00030541419982910156 s size 6797
Loaded 859 in 0.00032901763916015625 s
Saved 860 in 0.0002894401550292969 s size 6798
Loaded 860 in 0.00036716461181640625 s
Saved 861 in 0.0002872943878173828 s size 6798
Loaded 861 in 0.00034499168395996094 s
Saved 862 in 0.00027370452880859375 s size 6797
Loaded 862 in 0.00033020973205566406 s
Saved 863 in 0.0002837181091308594 s size 6797
Loaded 863 in 0.0003845691680908203 s
Saved 864 in 0.00028395652770996094 s size 6795
Loaded 864 in 0.00033092498779296875 s
Saved 865 in 0.00026917457580566406 s size 6790
Loaded 865 in 0.0003292560577392578 s
Saved 866 in 0.0002753734588623047 s size 6790
Lo

Saved 1006 in 0.000396728515625 s size 6786
Loaded 1006 in 0.0003693103790283203 s
Saved 1007 in 0.0003104209899902344 s size 6787
Loaded 1007 in 0.0003485679626464844 s
Saved 1008 in 0.0003256797790527344 s size 6785
Loaded 1008 in 0.00033354759216308594 s
Saved 1009 in 0.00035572052001953125 s size 6786
Loaded 1009 in 0.00035452842712402344 s
Saved 1010 in 0.00033545494079589844 s size 6787
Loaded 1010 in 0.00036334991455078125 s
Saved 1011 in 0.0002918243408203125 s size 6787
Loaded 1011 in 0.000339508056640625 s
Saved 1012 in 0.00028586387634277344 s size 6787
Loaded 1012 in 0.0003364086151123047 s
Saved 1013 in 0.00029158592224121094 s size 6787
Loaded 1013 in 0.00034999847412109375 s
Saved 1014 in 0.00034499168395996094 s size 6787
Loaded 1014 in 0.0003974437713623047 s
Saved 1015 in 0.00029087066650390625 s size 6787
Loaded 1015 in 0.0003387928009033203 s
Saved 1016 in 0.00028824806213378906 s size 6787
Loaded 1016 in 0.00033736228942871094 s
Saved 1017 in 0.0002911090850830078 

Saved 1152 in 0.0004508495330810547 s size 6796
Loaded 1152 in 0.0004215240478515625 s
Saved 1153 in 0.0003147125244140625 s size 6796
Loaded 1153 in 0.0003514289855957031 s
Saved 1154 in 0.0003566741943359375 s size 6797
Loaded 1154 in 0.0004220008850097656 s
Saved 1155 in 0.0003368854522705078 s size 6797
Loaded 1155 in 0.0003452301025390625 s
Saved 1156 in 0.0003104209899902344 s size 6797
Loaded 1156 in 0.0003516674041748047 s
Saved 1157 in 0.0002827644348144531 s size 6796
Loaded 1157 in 0.00034809112548828125 s
Saved 1158 in 0.0003323554992675781 s size 6796
Loaded 1158 in 0.0003418922424316406 s
Saved 1159 in 0.00028514862060546875 s size 6796
Loaded 1159 in 0.0003390312194824219 s
Saved 1160 in 0.00030159950256347656 s size 6796
Loaded 1160 in 0.0003561973571777344 s
Saved 1161 in 0.00027561187744140625 s size 6794
Loaded 1161 in 0.0003294944763183594 s
Saved 1162 in 0.0002770423889160156 s size 6792
Loaded 1162 in 0.00033354759216308594 s
Saved 1163 in 0.0002834796905517578 s 

Saved 1300 in 0.0003750324249267578 s size 6799
Loaded 1300 in 0.0004904270172119141 s
Saved 1301 in 0.0004534721374511719 s size 6799
Loaded 1301 in 0.0005576610565185547 s
Saved 1302 in 0.0004100799560546875 s size 6800
Loaded 1302 in 0.0005044937133789062 s
Saved 1303 in 0.0003178119659423828 s size 6799
Loaded 1303 in 0.00033926963806152344 s
Saved 1304 in 0.00027489662170410156 s size 6796
Loaded 1304 in 0.00032806396484375 s
Saved 1305 in 0.0002856254577636719 s size 6797
Loaded 1305 in 0.0003306865692138672 s
Saved 1306 in 0.000270843505859375 s size 6797
Loaded 1306 in 0.00032782554626464844 s
Saved 1307 in 0.0002741813659667969 s size 6797
Loaded 1307 in 0.0003287792205810547 s
Saved 1308 in 0.0002779960632324219 s size 6798
Loaded 1308 in 0.0003311634063720703 s
Saved 1309 in 0.0002772808074951172 s size 6799
Loaded 1309 in 0.00033020973205566406 s
Saved 1310 in 0.00028324127197265625 s size 6799
Loaded 1310 in 0.00032973289489746094 s
Saved 1311 in 0.0002884864807128906 s si

KeyboardInterrupt: 